In [47]:
import os
import re

import openslide
import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt
%matplotlib inline
import cPickle as pickle 

from boto.s3.connection import S3Connection
from boto.s3.key import Key

# pull in local code
import sys
sys.path.insert(0, '/Users/zgreyn/Documents/galvanize/projects/vegf/src')
import slideutils
import create_tiles


In [9]:
# create a pandas data frame where each row is a tile
df = pd.DataFrame()
df['key'] = np.nan
df['slide'] = np.nan
df['width'] = 64
df['height'] = 64
df['x'] = np.nan
df['y'] = np.nan

In [41]:
def populate_dataframe(slide, slide_name, df, tile_size=(64,64), level=0):
    '''
    This function breaks an openslide slide into tiles, discards empty tiles (just blank slide area),
    and records the location of each tile.  It only exists because I didn't originally save the locations...
    '''

    dimensions = slide.dimensions # (width, height)
    
    i = 0
    for x in xrange(0, dimensions[0], tile_size[0]):
        for y in xrange(0, dimensions[1], tile_size[1]):
            
            # create the tile (read_region takes location (upper left pixel), level, and size)
            tile = slide.read_region((x,y), level, tile_size)

            # don't look at blank tiles
            if not create_tiles.is_empty(tile):
                key = 'tiles/slide_' + slide_name + '_' + '{:06d}'.format(i) + '.tiff'
                record = pd.DataFrame([[key, slide_name, tile_size[0], tile_size[1], x, y]], columns=['key','slide','width','height','x','y'])
                df = df.append(record)
                
            # update the tile num
            i+=1

            if i%1000==0:
                print i

    return df

In [16]:
# get all the slides from the bucket
slide_names = []
for key in bucket.list():
    if key.name.endswith('.svs'):
        slide_names.append(key.name)

temp_slide_file = '../data/temp.svs'

In [44]:
# open each slide
for s3_key in slide_names:

    # save the file to a temporary local file
    k = bucket.get_key(s3_key)
    k.get_contents_to_filename(temp_slide_file)

    # open it with openslide
    slide = openslide.OpenSlide(temp_slide_file)

    # break the slide into tiles and save the metadata in the dataframe
    df = populate_dataframe(slide, s3_key[:-4], df)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [45]:
df.head()

,key,slide,width,height,x,y
0,something,other,64,64,0.0,0.0
0,tiles/slide_70229_000000.tiff,70229,64,64,0.0,0.0
0,tiles/slide_70229_000001.tiff,70229,64,64,0.0,64.0
0,tiles/slide_70229_000002.tiff,70229,64,64,0.0,128.0
0,tiles/slide_70229_000003.tiff,70229,64,64,0.0,192.0


In [48]:
pickle.dump(df, open( "../data/tiles_df.pkl", "wb" ))

In [54]:
df = df.reset_index()

In [56]:
df.drop(['index'], axis=1, inplace=True)

In [57]:
df.head()

,key,slide,width,height,x,y
0,something,other,64,64,0.0,0.0
1,tiles/slide_70229_000000.tiff,70229,64,64,0.0,0.0
2,tiles/slide_70229_000001.tiff,70229,64,64,0.0,64.0
3,tiles/slide_70229_000002.tiff,70229,64,64,0.0,128.0
4,tiles/slide_70229_000003.tiff,70229,64,64,0.0,192.0
